# COVID‑19 — Insights a partir de 34 tabelas derivadas (SQL → CSV)  
_Geração automática de gráficos e relatório (INSIGHTS.md)._

In [ ]:
# Imports essenciais
import os, glob, math, json
from pathlib import Path
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Renderização inline
%matplotlib inline

In [ ]:
# Descoberta robusta de caminhos para rodar do / ou de /notebooks
CWD = Path.cwd().resolve()
if CWD.name.lower() == "notebooks":
    BASE = CWD.parent
else:
    BASE = CWD

DATA_PROCESSED = BASE / "data" / "processed"
CHARTS_DIR     = BASE / "charts"
REPORTS_DIR    = BASE / "reports"
INSIGHTS_MD    = REPORTS_DIR / "INSIGHTS.md"

CHARTS_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

assert DATA_PROCESSED.exists(), f"Pasta não encontrada: {DATA_PROCESSED}"
csvs = sorted(DATA_PROCESSED.glob("*.csv"))
print(f\"CSV encontrados: {len(csvs)}\")
assert len(csvs) == 34, f"Esperados 34 CSVs, mas encontrei {len(csvs)}."

In [ ]:
# Leitura de todos os CSVs
dfs = {}
for f in csvs:
    try:
        df = pd.read_csv(f)
    except UnicodeDecodeError:
        df = pd.read_csv(f, encoding="latin-1")
    # Saneamento básico: padroniza nomes e tenta parsear datas por nome
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    for cand in ["date", "data", "dt", "report_date"]:
        if cand in df.columns:
            with pd.option_context("mode.chained_assignment", None):
                df[cand] = pd.to_datetime(df[cand], errors="coerce", utc=False)
    dfs[f.stem] = df

print("DataFrames carregados:", len(dfs))
list(dfs)[:10]

In [ ]:
def salvar_fig(nome_png: str):
    path = CHARTS_DIR / f"{nome_png}.png"
    plt.tight_layout()
    plt.savefig(path, dpi=150, bbox_inches="tight")
    print(f"[ok] gráfico salvo:", path)

In [ ]:
# Estratégias:
# - Se houver coluna de data + pelo menos 1 coluna numérica: série temporal (soma por data)
# - Se houver 'country'/'pais' e uma métrica numérica -> top 15 barras
# - Caso contrário, tenta um histograma de 1 métrica numérica

def possiveis_col_datas(cols):
    cs = [c for c in cols if any(tag in c for tag in ["date", "data", "dt"])]
    return cs

def possiveis_dim_pais(cols):
    for k in ["country", "pais", "country_region"]:
        if k in cols: return k
    return None

def col_numericas(df):
    return [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

gerados = 0
for key, df in dfs.items():
    try:
        cols = list(df.columns)
        datas = possiveis_col_datas(cols)
        dim_pais = possiveis_dim_pais(cols)
        nums = col_numericas(df)

        if len(datas) > 0 and len(nums) > 0:
            # série temporal: agrega por primeira coluna de data + primeira métrica
            dcol = datas[0]
            mcol = nums[0]
            tmp = df[[dcol, mcol]].dropna()
            # Caso existam muitas linhas por dia, agregue
            tmp = tmp.groupby(dcol, as_index=False)[mcol].sum()
            tmp = tmp.sort_values(dcol)
            ax = tmp.plot(x=dcol, y=mcol, title=f"{key} — {mcol} por data", figsize=(10,4))
            salvar_fig(f"{key}__{mcol}_por_data")
            plt.show()
            gerados += 1
            continue

        if dim_pais and len(nums) > 0:
            mcol = nums[0]
            tmp = (df[[dim_pais, mcol]]
                   .dropna()
                   .groupby(dim_pais, as_index=False)[mcol].sum()
                   .sort_values(mcol, ascending=False).head(15))
            ax = tmp.plot(kind="bar", x=dim_pais, y=mcol, title=f"{key} — Top 15 por {dim_pais}", figsize=(10,4))
            plt.xticks(rotation=45, ha="right")
            salvar_fig(f"{key}__top15_{dim_pais}")
            plt.show()
            gerados += 1
            continue

        if len(nums) > 0:
            mcol = nums[0]
            ax = df[mcol].dropna().plot(kind="hist", bins=30, title=f"{key} — distribuição de {mcol}", figsize=(10,4))
            salvar_fig(f"{key}__hist_{mcol}")
            plt.show()
            gerados += 1
    except Exception as e:
        print(f"[skip] {key}: {e}")

print(f"Gráficos gerados automaticamente: {gerados}")

In [ ]:
pngs = sorted(CHARTS_DIR.glob("*.png"))
bloco = [
    "# INSIGHTS — COVID-19",
    f"_Atualizado em: {datetime.now():%Y-%m-%d %H:%M}_",
    "",
    "## Índice de Gráficos",
]
for p in pngs:
    bloco.append(f"- {p.stem} — `charts/{p.name}`")

conteudo = "\n".join(bloco).strip()
if INSIGHTS_MD.exists():
    existente = INSIGHTS_MD.read_text(encoding="utf-8").strip()
    novo = existente + "\n\n---\n\n" + conteudo
else:
    novo = conteudo

INSIGHTS_MD.write_text(novo, encoding="utf-8")
print(f"[ok] INSIGHTS atualizado:", INSIGHTS_MD)

In [ ]:
print("Resumo final:")
print(" - CSVs lidos:", len(dfs))
print(" - PNGs no diretório charts/:", len(list(CHARTS_DIR.glob('*.png'))))

In [ ]:
# ▶️ CÉLULA FINAL — Gera reports/INSIGHTS.md com índice clicável + imagens embutidas
from pathlib import Path
from datetime import datetime
import os

# Descobre a raiz do projeto (funciona rodando da raiz ou de /notebooks)
CWD = Path.cwd().resolve()
BASE = CWD.parent if CWD.name.lower() == "notebooks" else CWD

CHARTS_DIR  = BASE / "charts"
REPORTS_DIR = BASE / "reports"
INSIGHTS_MD = REPORTS_DIR / "INSIGHTS.md"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

pngs = sorted(CHARTS_DIR.glob("*.png"))
if not pngs:
    raise SystemExit("Nenhum PNG encontrado em charts/. Gere os gráficos antes.")

linhas = []
linhas.append("# INSIGHTS — COVID-19")
linhas.append(f"_Atualizado em: {datetime.now():%Y-%m-%d %H:%M}_")
linhas.append("")
linhas.append("## Índice de Gráficos")
linhas.append("")

# Índice com links relativos a partir de reports/
for p in pngs:
    rel = os.path.relpath(p, REPORTS_DIR)
    linhas.append(f"- [{p.stem}]({rel})")

linhas.append("")
linhas.append("---")
linhas.append("")
linhas.append("## Galeria de Gráficos")
linhas.append("_Pré-visualização inline para GitHub e VS Code._")
linhas.append("")

# Galeria com imagens embutidas (links relativos corretos)
for p in pngs:
    rel = os.path.relpath(p, REPORTS_DIR)
    linhas.append(f"### {p.stem}")
    linhas.append(f"![{p.stem}]({rel})")
    linhas.append("")

INSIGHTS_MD.write_text("\n".join(linhas), encoding="utf-8")
print(f"[ok] INSIGHTS atualizado: {INSIGHTS_MD}")
print(f" - {len(pngs)} figuras referenciadas")